# Finding locations to establish temporary emergency facilities

Run this notebook to create a Decision Optimization model with Decision Optimization for Watson Studio and deploy the model using Watson Machine Learning.

The deployed model can later be accessed using the [Watson Machine Learning client library](https://wml-api-pyclient-dev-v4.mybluemix.net/) to find optimal location based on given constraints.

The model created here is a basic Decision Optimization model. The main purpose is to demonstrate creating a model and deploying using Watson Machine Learning. This model can and should be improved upon to include better constrainsts and provider better solutions.


## Steps

**Build and deploy model**

1. [Provision a Watson Machine Learning service](#provision-a-watson-machine-learning-service)
1. [Set up the Watson Machine Learning client library](#set-up-the-watson-machine-learning-client-library)
1. [Build the Decision Optimization model](#build-the-decision-optimization-model)
1. [Deploy the Decision Optimization model](#deploy-the-decision-optimization-model)

**Test the deployed model**

1. [Generate an API Key from the HERE Developer Portal](#generate-an-api-key-from-the-here-developer-portal)
1. [Query HERE API for Places](#query-here-api-for-places)
1. [Create and monitor a job to test the deployed model](#create-and-monitor-a-job-to-test-the-deployed-model)
1. [Extract and display solution](#extract-and-display-solution)


<br>

### Provision a Watson Machine Learning service

- If you do not have an IBM Cloud account, [register for a free trial account](https://cloud.ibm.com/registration).
- Log into [IBM Cloud](https://cloud.ibm.com/login)
- Create a [create a Watson Machine Learning instance](https://cloud.ibm.com/catalog/services/machine-learning)


<br>

### Set up the Watson Machine Learning client library

Install the [Watson Machine Learning client library](https://wml-api-pyclient-dev-v4.mybluemix.net/). This notebook uses the preview Python client based on v4 of Watson Machine Learning APIs. 

> **Important** Do not load both (V3 and V4) WML API client libraries into a notebook.


In [ ]:
# Uninstall the Watson Machine Learning client Python client based on v3 APIs

!pip uninstall watson-machine-learning-client -y

In [ ]:
# Install the WML client API v4

!pip install watson-machine-learning-client-V4

<br>

#### Create a client instance

Use your [Watson Machine Learning service credentials](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-get-wml-credentials.html) and update the next cell.

In [ ]:
# @hidden_cell

WML_API_KEY = '...'
WML_INSTANCE_ID = '...'
WML_URL = 'https://us-south.ml.cloud.ibm.com'

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
# Instantiate a client using credentials
wml_credentials = {
  'apikey': WML_API_KEY,
  'instance_id': WML_INSTANCE_ID,
  'url': WML_URL
}

client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
client.version

<br>

### Build the Decision Optimization model

- The Decision Optimization model will be saved to a `model.py` file in a subdirectory (i.e., `model/`) of the current working directory.
- The model will be placed in a tar archive and uploaded to Watson Machine Learning.

Set up variables for model and deployment

In [ ]:
import os


model_dir = 'model'
model_file = 'model.py'
model_path = '{}/{}'.format(model_dir, model_file)

model_tar = 'model.tar.gz'
model_tar_path = '{}/{}'.format(os.getcwd(), model_tar)

model_name = 'DO_HERE_DEMO'
model_desc = 'Finding locations for short-term emergency facilities'

deployment_name = 'DO_HERE_DEMO Deployment'
deployment_desc = 'Deployment of DO_HERE_DEMO model'


print(model_path)
print(model_tar_path)

<br>

#### Create the model.py in a model subdirectory

Use the  `mkdir` and `write_file` commands to create the subdirectory and write the model code to a file. 


In [ ]:
%mkdir $model_dir

In [ ]:
%%writefile $model_path

from docplex.util.environment import get_environment
from os.path import splitext
import pandas
from six import iteritems
import json


def get_all_inputs():
    '''Utility method to read a list of files and return a tuple with all
    read data frames.
    Returns:
        a map { datasetname: data frame }
    '''
    result = {}
    env = get_environment()
    for iname in [f for f in os.listdir('.') if splitext(f)[1] == '.csv']:
        with env.get_input_stream(iname) as in_stream:
            df = pandas.read_csv(in_stream)
            datasetname, _ = splitext(iname)
            result[datasetname] = df
    return result

def write_all_outputs(outputs):
    '''Write all dataframes in ``outputs`` as .csv.

    Args:
        outputs: The map of outputs 'outputname' -> 'output df'
    '''
    for (name, df) in iteritems(outputs):
        if isinstance(df, pandas.DataFrame):
            csv_file = '%s.csv' % name
            print(csv_file)
            with get_environment().get_output_stream(csv_file) as fp:
                if sys.version_info[0] < 3:
                    fp.write(df.to_csv(index=False, encoding='utf8'))
                else:
                    fp.write(df.to_csv(index=False).encode(encoding='utf8'))
        elif isinstance(df, str):
            txt_file = '%s.txt' % name 
            with get_environment().get_output_stream(txt_file) as fp:
                fp.write(df.encode(encoding='utf8'))
                
    if len(outputs) == 0:
        print('Warning: no outputs written')


In [ ]:
%%writefile -a $model_path

from docplex.mp.model import Model
from statistics import mean


def get_distance(routes_df, start, destination):
    s = getattr(start, 'geocode', start)
    d = getattr(destination, 'geocode', destination)
    row = routes_df.loc[
        (routes_df['start'] == s) &
        (routes_df['destination'] == d)
    ]
    return row['distance'].values[0]


def build_and_solve(places_df, routes_df, number_sites=3):
    print('Building and solving model')
    
    mean_dist = mean(routes_df['distance'].unique())
    p_only = places_df.loc[places_df['is_medical'] == False]
    h_only = places_df.loc[places_df['is_medical'] == True]
    
    places = list(p_only.itertuples(name='Place', index=False))

    postal_codes = p_only['postal_code'].unique()
    hospital_geocodes = h_only['geocode'].unique()

    mdl = Model(name='temporary emergency sites')
    
    ## decision variables
    places_vars = mdl.binary_var_dict(places, name='is_place')
    postal_link_vars = mdl.binary_var_matrix(postal_codes, places, 'link')
    hosp_link_vars = mdl.binary_var_matrix(hospital_geocodes, places, 'link')

    ## objective function
    # minimize hospital distances
    h_total_distance = mdl.sum(hosp_link_vars[h, p] * abs(mean_dist - get_distance(routes_df, h, p)) for h in hospital_geocodes for p in places)
    mdl.minimize(h_total_distance)

    ## constraints
    # match places with their correct postal_code
    for p in places:
        for c in postal_codes:
            if p.postal_code != c:
                mdl.add_constraint(postal_link_vars[c, p] == 0, 'ct_forbid_{0!s}_{1!s}'.format(c, p))

    # # each postal_code should have one only place
    # mdl.add_constraints(
    #     mdl.sum(postal_link_vars[c, p] for p in places) == 1 for c in postal_codes
    # )

    # # each postal_code must be associated with a place
    # mdl.add_constraints(
    #     postal_link_vars[c, p] <= places_vars[p] for p in places for c in postal_codes
    # )

    # solve for 'number_sites' places
    mdl.add_constraint(mdl.sum(places_vars[p] for p in places) == number_sites)

    ## model info
    mdl.print_information()
    stats = mdl.get_statistics()

    ## model solve
    mdl.solve(log_output=True)
    details = mdl.solve_details

    status = '''
    Model stats
      number of variables: {}
      number of constraints: {}

    Model solve
      time (s): {}
      status: {}
    '''.format(
        stats.number_of_variables,
        stats.number_of_constraints,
        details.time,
        details.status
    )

    possible_sites = [p for p in places if places_vars[p].solution_value == 1]

    return possible_sites, status


In [ ]:
%%writefile -a $model_path

import pandas


def run():
    # Load CSV files into inputs dictionary
    inputs = get_all_inputs()

    places_df = inputs['places']    
    routes_df = inputs['routes']

    site_suggestions, status = build_and_solve(places_df, routes_df)
    solution_df = pandas.DataFrame(site_suggestions)

    outputs = {
        'solution': solution_df,
        'status': status
    }

    # Generate output files
    write_all_outputs(outputs)
    
    
run()


<br>

#### Create the model tar archive

Use the `tar` command to create a tar archive with the model file.


In [ ]:
import tarfile

def reset(tarinfo):
    tarinfo.uid = tarinfo.gid = 0
    tarinfo.uname = tarinfo.gname = 'root'
    return tarinfo

tar = tarfile.open(model_tar, 'w:gz')
tar.add(model_path, arcname=model_file, filter=reset)
tar.close()

<br>

### Deploy the Decision Optimization model

Store model in Watson Machine Learning with:

- the tar archive previously created,
- metadata including the model type and runtime


In [ ]:
# All available meta data properties 

client.repository.ModelMetaNames.show()

In [ ]:
# All available runtimes

client.runtimes.list(pre_defined=True)

<br>

#### Upload the model to Watson Machine Learning

Configure the model metadata and set the model type (i.e., `do-docplex_12.9`) and runtime (i.e., `do_12.9`)


In [ ]:
import os

model_metadata = {
    client.repository.ModelMetaNames.NAME: model_name,
    client.repository.ModelMetaNames.DESCRIPTION: model_desc,
    client.repository.ModelMetaNames.TYPE: 'do-docplex_12.9',
    client.repository.ModelMetaNames.RUNTIME_UID: 'do_12.9'
}

model_details = client.repository.store_model(model=model_tar_path, meta_props=model_metadata)

model_uid = client.repository.get_model_uid(model_details)

print('Model GUID: {}'.format(model_uid))

<br>

#### Create a deployment 

Create a batch deployment for the model, providing deployment metadata and model UID.


In [ ]:
deployment_metadata = {
    client.deployments.ConfigurationMetaNames.NAME: deployment_name,
    client.deployments.ConfigurationMetaNames.DESCRIPTION: deployment_desc,
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.COMPUTE: {'name': 'S', 'nodes': 1}
}

deployment_details = client.deployments.create(model_uid, meta_props=deployment_metadata)

deployment_uid = client.deployments.get_uid(deployment_details)

print('Deployment GUID: {}'.format(deployment_uid))

<br>

**Congratulations!** The model has been succesfully deployed. Please make a note of the deployment UID.

<br>

## Test the deployed model

### Generate an API Key from the HERE Developer Portal

To test your deployed model using actual data from HERE Location services, you'll need an API key.   

Follow the instructions outlined in the [HERE Developer Portal](https://developer.here.com/sign-up) to [generate an API key](https://developer.here.com/documentation/authentication/dev_guide/topics/api-key-credentials.html).

Use your [HERE.com API key](https://developer.here.com/sign-up) and update the next cell.


In [ ]:
# @hidden_cell

HERE_APIKEY = '...'


<br>

Set up helper functions to query HERE APIs

In [ ]:
import re
import requests

geocode_endpoint = 'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={api_key}'
browse_endpoint = 'https://browse.search.hereapi.com/v1/browse?categories=%s&at=%s&apiKey=%s'
matrix_routing_endpoint = 'https://matrix.route.ls.hereapi.com/routing/7.2/calculatematrix.json?mode=%s&summaryAttributes=%s&apiKey=%s'

coordinates_regex = '^[-+]?([1-8]?\d(\.\d+)?|90(\.0+)?),\s*[-+]?(180(\.0+)?|((1[0-7]\d)|([1-9]?\d))(\.\d+)?)$'

def is_geocode (location):
    geocode = None
    if isinstance(location, str):
        l = location.split(',')
        if len(l) == 2:
            geocode = '{},{}'.format(l[0].strip(), l[1].strip())
    elif isinstance(location, list) and len(location) == 2:
        geocode = ','.join(str(l) for l in location)
      
    if geocode is not None and re.match(coordinates_regex, geocode):
        return [float(l) for l in geocode.split(',')]
    else:
        return False

    
def get_geocode (address):
    g = is_geocode(address)

    if not g:
        url = geocode_endpoint.format(address=address, api_key=HERE_APIKEY)
        response = requests.get(url)
        
        if response.ok:
            jsonResponse = response.json()
            position = jsonResponse['items'][0]['position']
            g = [position['lat'], position['lng']]
        else:
            print(response.text)

    return g


def get_browse_url (location, categories, limit=25):
    categories = ','.join(c for c in categories)
    geocode = get_geocode(location)
    coordinates = ','.join(str(g) for g in geocode)

    browse_url = browse_endpoint % (
        categories,
        coordinates,
        HERE_APIKEY
    )

    if limit > 0:
        browse_url = '{}&limit={}'.format(browse_url, limit)
  
    return browse_url


def browse_places (location, categories=[], results_limit=100):
    places_list = []

    browse_url = get_browse_url(location, categories, limit=results_limit)
    response = requests.get(browse_url)

    if response.ok:
        json_response = response.json()
        places_list = json_response['items']
    else:
        print(response.text)

    return places_list


def get_places_nearby (location, categories=[], results_limit=100, max_distance_km=50):
    places_list = browse_places(location, categories=categories, results_limit=results_limit)

    filtered_places = []

    for p in places_list:
        if p['distance'] <= max_distance_km * 1000:
            filtered_places.append(Place(p))

    return filtered_places


def get_hospitals_nearby (location, results_limit=100, max_distance_km=50):
    h_cat = ['800-8000-0159']
    hospitals_list = browse_places(location, categories=h_cat, results_limit=results_limit)

    filtered_hospitals = []

    for h in hospitals_list:
        if h['distance'] <= max_distance_km * 1000:
            filtered_hospitals.append(Place(h, is_medical=True))

    return filtered_hospitals


def get_matrix_routing_url ():
    route_mode = 'shortest;car;traffic:disabled;'
    summary_attributes = 'routeId,distance'

    matrix_routing_url = matrix_routing_endpoint % (
        route_mode,
        summary_attributes,
        HERE_APIKEY
    )

    return matrix_routing_url


def get_route_summaries (current_geocode, places, hospitals):
    # Request should not contain more than 15 start positions
    num_starts = 15

    postal_codes_set = set()
    postal_codes_geocodes = []
    places_waypoints = {}

    for i, p in enumerate(places):
        if p.postal_code:
            postal_codes_set.add('{}:{}'.format(p.postal_code, p.country))
        places_waypoints['destination{}'.format(i)] = p.geocode

    for p in postal_codes_set:
        geocode = get_geocode(p)
        postal_codes_geocodes.append({
            'postal_code': p.split(':')[0],
            'geocode': ','.join(str(g) for g in geocode)
        })

    current = {
        'geocode': ','.join(str(g) for g in current_geocode)
    }

    start_geocodes = [current] + postal_codes_geocodes + [h.to_dict() for h in hospitals]
    start_coords = [
        start_geocodes[i:i+num_starts] 
        for i in range(0, len(start_geocodes), num_starts)
    ]

    route_summaries = []
    matrix_routing_url = get_matrix_routing_url()

    for sc in start_coords:
        start_waypoints = {}
        for i, s in enumerate(sc):
            start_waypoints['start{}'.format(i)] = s['geocode']

        coords = {**start_waypoints, **places_waypoints}
        response = requests.post(matrix_routing_url, data = coords)

        if not response.ok:
            print(response.text)
        else:
            json_response = response.json()
            for entry in json_response['response']['matrixEntry']:
                start_geocode = start_waypoints['start{}'.format(entry['startIndex'])]
                dest_geocode = places_waypoints[
                    'destination{}'.format(entry['destinationIndex'])
                ]

                for s in sc:
                    if 'address' not in s and 'postal_code' in s and s['geocode'] == start_geocode:
                        route_summaries.append({
                            'start': s['postal_code'],
                            'destination': dest_geocode,
                            'distance': entry['summary']['distance'],
                            'route_id': entry['summary']['routeId']
                        })
                        break

                route_summaries.append({
                    'start': start_geocode,
                    'destination': dest_geocode,
                    'distance': entry['summary']['distance'],
                    'route_id': entry['summary']['routeId']
                })

    return route_summaries


<br>

Define a Place class


In [ ]:
class Place(object):
    def __init__(self, p, is_medical=False):
        self.id = p['id']
        self.title = p['title']
        self.address = p['address']['label'] if 'label' in p['address'] else p['address']
        self.postal_code = p['address']['postalCode'] if 'postalCode' in p['address'] else p['postal_code']
        self.distance = p['distance']
        self.primary_category = p['categories'][0]['id'] if 'categories' in p else p['primary_category']
        self.geocode = '{},{}'.format(p['position']['lat'], p['position']['lng']) if 'position' in p else p['geocode']
        self.country = p['address']['countryCode'] if 'countryCode' in p['address'] else p['country']
        self.is_medical = p['is_medical'] if 'is_medical' in p else is_medical
        if isinstance(self.is_medical, str):
            self.is_medical = self.is_medical.lower() in ['true', '1']
    
    def to_dict(self):
        location = self.geocode.split(',')
        return({
          'id': self.id,
          'title': self.title,
          'address': self.address,
          'postal_code': self.postal_code,
          'distance': self.distance,
          'primary_category': self.primary_category,
          'geocode': self.geocode,
          'country': self.country,
          'is_medical': self.is_medical
        })
    
    def __str__(self):
        return self.address


<br>

### Query HERE API for Places

Use the HERE API to get a list of Places in the vicinity of an address

Example of `Place` entity returned by HERE API:
```json
  {
    'title': 'Duane Street Hotel',
    'id': 'here:pds:place:840dr5re-fba2a2b91f944ee4a699eea7556896bd',
    'resultType': 'place',
    'address': {
      'label': 'Duane Street Hotel, 130 Duane St, New York, NY 10013, United States',
      'countryCode': 'USA',
      'countryName': 'United States',
      'state': 'New York',
      'county': 'New York',
      'city': 'New York',
      'district': 'Tribeca',
      'street': 'Duane St',
      'postalCode': '10013',
      'houseNumber': '130'
    },
    'position': { 'lat': 40.71599, 'lng': -74.00735 },
    'access': [ { 'lat': 40.71608, 'lng': -74.00728 } ],
    'distance': 161,
    'categories': [
      { 'id': '100-1000-0000' },
      { 'id': '200-2000-0000' },
      { 'id': '500-5000-0000' },
      { 'id': '500-5000-0053' },
      { 'id': '500-5100-0000' },
      { 'id': '700-7400-0145' }
    ],
    'foodTypes': [ { 'id': '101-000' } ],
    'contacts': [ ],
    'openingHours': [
      {
        'text': [ 'Mon-Sun: 00:00 - 24:00' ],
        'isOpen': true,
        'structured': [
          {
            'start': 'T000000',
            'duration': 'PT24H00M',
            'recurrence': 'FREQ:DAILY;BYDAY:MO,TU,WE,TH,FR,SA,SU'
          }
        ]
      }
    ]
  }
```

In [ ]:
address = 'New York, NY'
max_results = 20

# HERE Place Category System
# https://developer.here.com/documentation/geocoding-search-api/dev_guide/topics-places/places-category-system-full.html
places_categories = ['500-5000'] # Hotel-Motel

current_geocode = get_geocode(address)

places = get_places_nearby(
    current_geocode,
    categories=places_categories,
    results_limit=max_results
)

hospitals = get_hospitals_nearby(
    current_geocode,
    results_limit=3
)

print('Places:')
for p in places:
    print(p)

print('\nHospitals:')
for h in hospitals:
    print(h)

<br>

### Create and monitor a job to test the deployed model

Create a payload containing places data received from HERE


In [ ]:
import pandas as pd

places_df = pd.DataFrame.from_records([p.to_dict() for p in (places + hospitals)])

places_df.head()

In [ ]:
route_summaries = get_route_summaries(current_geocode, places, hospitals)

routes_df = pd.DataFrame.from_records(route_summaries)
routes_df.drop_duplicates(keep='last', inplace=True)

routes_df.head()


In [ ]:
solve_payload = {
    client.deployments.DecisionOptimizationMetaNames.INPUT_DATA: [
        { 'id': 'places.csv', 'values' : places_df },
        { 'id': 'routes.csv', 'values' : routes_df }
    ],
    client.deployments.DecisionOptimizationMetaNames.OUTPUT_DATA: [
        { 'id': '.*\.csv' },
        { 'id': '.*\.txt' }
    ]
}

<br>

Submit a new job with the payload and deployment.  
Set the UID of the deployed model.


In [ ]:
# deployment_uid = '...'

In [ ]:
job_details = client.deployments.create_job(deployment_uid, solve_payload)
job_uid = client.deployments.get_job_uid(job_details)

print('Job UID: {}'.format(job_uid))

Display job status until it is completed.

The first job of a new deployment might take some time as a compute node must be started.

In [ ]:
from time import sleep

while job_details['entity']['decision_optimization']['status']['state'] not in ['completed', 'failed', 'canceled']:
    print(job_details['entity']['decision_optimization']['status']['state'] + '...')
    sleep(3)
    job_details=client.deployments.get_job_details(job_uid)

print(job_details['entity']['decision_optimization']['status']['state'])

In [ ]:
# job_details
job_details['entity']['decision_optimization']['status']

<br>

### Extract and display solution

Display the output solution.

In [ ]:
import base64

output_data = job_details['entity']['decision_optimization']['output_data']
solution = None
stats = None

for i, d in enumerate(output_data):
    if d['id'] == 'solution.csv':
        solution = pd.DataFrame(output_data[i]['values'], 
                        columns = job_details['entity']['decision_optimization']['output_data'][0]['fields'])
    else:
        stats = base64.b64decode(output_data[i]['values'][0][0]).decode('utf-8')

print(stats)
solution.head()


<br>

Check out the online documentation at <a href="https://dataplatform.cloud.ibm.com/docs" target="_blank" rel="noopener noreferrer">https://dataplatform.cloud.ibm.com/docs</a> for more samples, tutorials and documentation. 


<br>

## Helper functions

See `watson-machine-learning-client(V4)` Python library documentation for more info on the API:
https://wml-api-pyclient-dev-v4.mybluemix.net/


In [ ]:
## List models
def list_models(wml_client):
    wml_client.repository.list_models()

    
## List deployments
def list_deployments(wml_client):
    wml_client.deployments.list()
    

## Delete a model
def delete_model(wml_client, model_uid):
    wml_client.repository.delete(model_uid)
    

## Delete a deployment
def delete_deployment(wml_client, deployment_uid):
    wml_client.deployments.delete(deployment_uid)

    
## Get details of all models
def details_all_models(wml_client):
    return wml_client.repository.get_model_details()['resources']


## Get details of all deployments
def details_all_deployments(wml_client):
    return wml_client.deployments.get_details()['resources']


# Find model using model name
def get_models_by_name(wml_client, model_name):
    all_models = wml_client.repository.get_model_details()['resources']
    models = [m for m in all_models if m['entity']['name'] == model_name]
    return models


# Find deployment using deployment name
def get_deployments_by_name(wml_client, deployment_name):
    all_deployments = wml_client.deployments.get_details()['resources']
    deployments = [d for d in all_deployments if d['entity']['name'] == deployment_name][0]
    return deployments


In [ ]:
delete_deployment(client, deployment_uid)

In [ ]:
delete_model(client, model_uid)

In [ ]:
list_deployments(client)

In [ ]:
list_models(client)